### Install pre-requisites

In [ ]:
%system python3 -m pip install pymilvus confluent-kafka astropy scikit-image

### Init Milvus

This is preliminary stuff 

important PRESTO.CRT and PORT

In [ ]:
rc = %system echo QUIT | openssl s_client -showcerts -connect watsonxdata:8443 | \
        awk '/-----BEGIN CERTIFICATE-----/ {p=1}; p; /-----END CERTIFICATE-----/ {p=0}' > ./presto.crt 

In [ ]:
import glob
import numpy as np

from pymilvus import(
    Milvus,
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema,
    utility,
    MilvusClient
)

from astropy.io import fits
from skimage.transform import resize

def connect_to_milvus() :
    
    host         = 'eu-de.services.cloud.techzone.ibm.com'
    port         = 25782        # <----------- write the external Milvus port here. You get this in your reservation
    user         = 'ibmlhadmin'
    key          = 'password'
    server_pem_path = 'presto.crt'
    connections.connect(alias='default',
                       host=host,
                       port=port,
                       user=user,
                       password=key,
                       server_pem_path=server_pem_path,
                       server_name='watsonxdata',
                       secure=True)  

def create_collection():
    
    utility.drop_collection("image_embeddings")
    
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="file_path", dtype=DataType.VARCHAR, max_length=128),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=16600)       
    ]
    schema = CollectionSchema(fields, "Embedding of FITS image file")
    
    fits_coll = Collection("image_embeddings", schema)

    index_params = {
            'metric_type':'L2',
            'index_type':"IVF_FLAT",
            'params':{"nlist":2048}
    }
    fits_coll.create_index(field_name="embedding", index_params=index_params)

    fits_coll.flush()
    
    return(fits_coll)

def load_fits_file(file_path) :
    
    with fits.open(file_path) as hdul:
   
        image_data = hdul[0].data
        image_resized = resize(image_data, (166, 100), mode='reflect')

    return (image_resized ) 

def generate_embedding(image_data) : 
    
    embedding = image_data.flatten()
    embedding = embedding / np.linalg.norm(embedding)  
    
    return embedding
    
def insert_embedding(fits_coll, file_path, embedding):

    fits_coll.insert([[file_path], [embedding]])
    fits_coll.load()

def initialize_collection():
    fits_coll = create_collection()
    file_paths = glob.glob("./images/m31*.FITS")
    for image_file in sorted(file_paths):
        print("Inserting file: ", image_file)
        image_data = load_fits_file(image_file)
        embedding_vector = generate_embedding(image_data)
        insert_embedding(fits_coll, image_file, embedding_vector)
    return fits_coll

Now, the action begins

In [ ]:
connect_to_milvus()
fits_coll = initialize_collection()
connections.disconnect(alias="default")

### Init watsonx.data

Preliminary stuff

In [ ]:
import base64
import prestodb
import glob

import numpy as np
import pandas as pd

def connect_to_watsonxdata() :

    # Connection Parameters
    userid     = 'ibmlhadmin'
    password   = 'password'
    hostname   = 'watsonxdata'
    port       = '8443'
    catalog    = 'tpch'
    schema     = 'tiny'
    certfile   = "/certs/lh-ssl-ts.crt"

    # Connect Statement
    try:
        wxdconnection = prestodb.dbapi.connect(
                host=hostname,
                port=port,
                user=userid,
                catalog=catalog,
                schema=schema,
                http_scheme='https',
                auth=prestodb.auth.BasicAuthentication(userid, password)
        )
        if (certfile != None):
            wxdconnection._http_session.verify = certfile
        print("Connection successful")
        return wxdconnection
    except Exception as e:
        print("Unable to connect to the database.")
        print(repr(e))

def create_staging_table(wxdconnection) :

    cursor = wxdconnection.cursor()

    sql = '''
        CREATE SCHEMA IF NOT EXISTS 
            iceberg_data.fits 
        WITH (location = 's3a://iceberg-bucket/fits') 
    '''
    try:
        cursor.execute(sql)
    except Exception as err:
        print(repr(err))
    
    sql = '''
        DROP TABLE IF EXISTS 
            iceberg_data.fits."fits-images"
    '''
    try:
        cursor.execute(sql)
    except Exception as err:
        print(repr(err))
        
    sql = '''
        CREATE TABLE 
            iceberg_data.fits."fits-images" (
                filename   VARCHAR,
                filebytes  VARCHAR
            )
    '''
    try:
        cursor.execute(sql)
    except Exception as err:
        print(repr(err))

    cursor.close()

def insert_file(wxdconnection, image_file):

    with open(image_file, 'rb') as file:
        file_content = file.read()

    encoded_file_content = base64.b64encode(file_content).decode('utf-8')

    cursor = wxdconnection.cursor()

    # I know this is a crime
    sql = f'''
        INSERT INTO iceberg_data.fits."fits-images" (filename, filebytes)
        VALUES ( '{image_file}', '{encoded_file_content}' )
    '''  

    try:
        cursor.execute(sql)
        wxdconnection.commit() 
    except Exception as err:
        print(f"Error executing SQL: {repr(err)}")
    finally:
        cursor.close()  

Now the action begins

In [ ]:
wxdconnection = connect_to_watsonxdata()

create_staging_table(wxdconnection)

file_paths = glob.glob("./images/m31*.FITS")
for image_file in sorted(file_paths):
        print("Inserting file: ", image_file)
        insert_file(wxdconnection,image_file)